<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_visual_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a visual puzzle in PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

Let's start by installing `borb`

In [5]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We are going to be using a custom font for the title of this PDF. So let's start by downloading it;

In [6]:
from borb.pdf.canvas.font.simple_font.true_type_font import TrueTypeFont  
from borb.pdf.canvas.font.font import Font

# Download Font
import requests
with open('IndieFlower-Regular.ttf', 'wb') as ffh:
  ffh.write(requests.get("https://github.com/google/fonts/blob/main/ofl/indieflower/IndieFlower-Regular.ttf?raw=true", allow_redirects=True).content)

In [7]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.layout.image.image import Image
from borb.pdf.canvas.layout.table.fixed_column_width_table import FixedColumnWidthTable

# Font
from borb.pdf.canvas.color.color import HexColor

# Python
from decimal import Decimal
from pathlib import Path  
import typing
import random

# create Document
doc: Document = Document()

page: Page = Page()
doc.add_page(page)

# set PageLayout
layout: PageLayout = SingleColumnLayout(page)

# add title
layout.add(Paragraph('Odd one out', 
                     font_color=HexColor('#19647E'),
                      font=TrueTypeFont.true_type_font_from_file(Path("IndieFlower-Regular.ttf")),
                      font_size=Decimal(20)))
  
# add explanation
layout.add(Paragraph('Each animal is displayed twice, except for one. Can you find the odd one out?',
                     font_color=HexColor('#28AFB0')))

# create array containing 10 animals (adding each animal twice)
cells: typing.List[str] = ["https://icons.iconarchive.com/icons/icojam/animals/64/01-bird-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-bull-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-cat-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-cow-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-dog-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-duck-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-elephant-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-fish-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-horse-icon.png",
                           "https://icons.iconarchive.com/icons/icojam/animals/64/01-ladybug-icon.png",
                            ] * 2

# remove random animal (thus creating the odd one out)
cells.pop(random.randint(0, len(cells)-1))

# populate remainder with None
for _ in range(0, 49 - len(cells)):
  cells.append(None)

# shuffle
random.shuffle(cells)

# build Table using cells
table:FixedColumnWidthTable = FixedColumnWidthTable(number_of_rows=7, number_of_columns=7)
for s in cells:
  if s == None:
    table.add(Paragraph(' '))
  else:
    table.add(Image(s, 
                    width=Decimal(32), 
                    height=Decimal(32)))

table.set_padding_on_all_cells(Decimal(5), Decimal(5), Decimal(5), Decimal(5))
table.no_borders()

# add to PageLayout
layout.add(table)

# write Document
with open("output.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, doc)